# A) Initial Processing

### Create Snow Depth Rasters
Script to run terminal commands to create inititial snow depth grids. The process checks whether the rasters are the same size then will proceed to either A) warp the snow depth grid and subtract the bare earth or B) proceed to subtract the bare earth from snow depth. 
#### Troubleshooting
If issues with subprocess command, this can be run directly from the terminal. The "shell = True" solved my problems as I need to activate my anaconda environment to succesfully run GDAL commands. 

In [1]:
from osgeo import gdal
import numpy
import os
import rasterio as rio
import subprocess
import sys
from subprocess import Popen, PIPE, CalledProcessError

In [2]:
#function to subtract the bare earth from the snow on raster to create snow depth grid
def gdal_calc(bareearth_raster, snowon_raster, gdalcalc,snowdepth_name):
    snowdepth_directory = os.path.join("outputs", snowdepth_name)
    print(snowdepth_directory)
     #if this file already exists, gdal_calc will not run 
    if os.path.exists(snowdepth_directory): 
        os.remove(snowdepth_directory)
    cmd_calc = 'python {} -A {} -B {} --outfile={} --calc="A-B"'.format(gdalcalc, snowon_raster, bareearth_raster,snowdepth_directory)
    #this will print the gdal calc processing
    with Popen(cmd_calc, stdout=PIPE, bufsize=1, universal_newlines=True) as p:
        for line in p.stdout:
            print(line, end='') # process line here
    subprocess.run(cmd_calc, shell=True) #should exit with code 0 if succesful 

In [3]:
#function to first warp the "snowon_raster" to the bare earth if different extents
def gdal_warp(bareearth_raster, snowon_raster, be_read):
    #create the file name by adding warp to the snowon raster file name
    print("creating bounding box..")
    min_x=be_read.bounds[0]
    max_x=be_read.bounds[2]
    min_y=be_read.bounds[1]
    max_y=be_read.bounds[3]
    warp_name= str(snowon_raster)[:-4] + "_warp.tif"
    print(warp_name)
    #if this file already exists, gdawarp will not run 
    if os.path.exists(warp_name): 
        os.remove(warp_name)
    #creating command statement   
    cmd_warp = 'gdalwarp -te {} {} {} {} {} {}'.format(min_x, min_y, max_x, max_y, snowon_raster, warp_name)
    #enabling subprocess to print out live processing
    with Popen(cmd_warp, stdout=PIPE, bufsize=1, universal_newlines=True) as p:
        for line in p.stdout: 
            print(line, end='')
    #call the gdal_warp command
    subprocess.run(cmd_warp, shell=True)
    #return the output file
    return warp_name

In [4]:
#file directrories, update as needed
bareearth_raster = r"inputs\EGM_BE_wgs84_1m.tif"
snowon_raster =  r"inputs\EGM_P01_STM_coreg.tif"
gdalcalc =  r"inputs\gdal_calc.py"

#based on snow_on raster name, may need to update if your naming convention differs
snowdepth_name = str(snowon_raster)[7:-14] + "_SD.tif"
print(snowdepth_name)

#read in the rasters
be_read = rio.open(bareearth_raster)
snowon_read = rio.open(snowon_raster)

#Check if the rasters are different sizes
if be_read.shape != snowon_read.shape:
    #Warp the snowon raster then reassign the variable
    snowon_raster = gdal_warp(bareearth_raster, snowon_raster, be_read)

#subtract the bare earth from the snow on raster and save in outputs folder
gdal_calc(bareearth_raster, snowon_raster, gdalcalc,snowdepth_name)

EGM_P01_SD.tif
creating bounding box..
inputs\EGM_P01_STM_coreg_warp.tif
Creating output file that is 11164P x 16434L.
Processing inputs\EGM_P01_STM_coreg.tif [1/1] : 0Using internal nodata values (e.g. -9999) for image inputs\EGM_P01_STM_coreg.tif.
Copying nodata values from source inputs\EGM_P01_STM_coreg.tif to destination inputs\EGM_P01_STM_coreg_warp.tif.
...10...20...30...40...50...60...70...80...90...100 - done.
outputs\EGM_P01_SD.tif
0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.. 0.